In [1]:
!pip install --upgrade --user --quiet google-cloud-aiplatform google-cloud-bigquery

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 53.6 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [17]:
# Get project ID
PROJECT_ID = ! gcloud config get-value project
PROJECT_ID = PROJECT_ID[0]
LOCATION = "us-central1" # @param {type:"string"}
DATASET_LOCATION = "US"
print(PROJECT_ID)

qwiklabs-gcp-04-ee8165cd97c8


In [3]:
from google.cloud import aiplatform
aiplatform.init(project=PROJECT_ID, location=LOCATION)

print("Initialized")

Initialized


In [18]:
from google.cloud import bigquery
from google.api_core.exceptions import NotFound

# BigQuery dataset/table configuration.
DATASET_ID_REQUESTED = "genai-skills-workshop"
# BigQuery converts hyphens to underscores in dataset IDs.
DATASET_ID = DATASET_ID_REQUESTED.replace("-", "_")
TABLE_ID = "aurora_bay_faqs"
GCS_URI = "gs://labs.roitraining.com/aurora-bay-faqs/aurora-bay-faqs.csv"

# Reuse the same project inferred earlier in the notebook.
bq_client = bigquery.Client(project=PROJECT_ID)

# Build dataset reference anchored to the desired location.
dataset_ref = bigquery.Dataset(f"{PROJECT_ID}.{DATASET_ID}")
dataset_ref.location = DATASET_LOCATION

# Create the dataset if it does not yet exist.
try:
  bq_client.get_dataset(dataset_ref)
  print(f"Dataset `{PROJECT_ID}.{DATASET_ID}` already exists.")
except NotFound:
  bq_client.create_dataset(dataset_ref)
  print(f"Created dataset `{PROJECT_ID}.{DATASET_ID}` in {DATASET_LOCATION}.")


Created dataset `qwiklabs-gcp-04-ee8165cd97c8.genai_skills_workshop` in US.


In [32]:
# Fully qualified table name we will load the CSV into.
table_id = f"{PROJECT_ID}.{DATASET_ID}.{TABLE_ID}"

# Configure the ingestion job to use CSV autodetect and to overwrite any prior data.
load_job_config = bigquery.LoadJobConfig(
  source_format=bigquery.SourceFormat.CSV,
  skip_leading_rows=1,
  autodetect=True,
  write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE,
)

# Launch the load job from the public GCS bucket into BigQuery.
load_job = bq_client.load_table_from_uri(GCS_URI, table_id, job_config=load_job_config)
print(f"Starting load job {load_job.job_id}")

# Wait for the load job to complete and fetch metadata for confirmation.
load_job.result()
table = bq_client.get_table(table_id)
print(f"Loaded {table.num_rows} rows into {table.full_table_id}.")


Starting load job 689c228b-9a8f-4cdc-a8e7-ce5c3a607078
Loaded 50 rows into qwiklabs-gcp-04-ee8165cd97c8:genai_skills_workshop.aurora_bay_faqs.


In [10]:
from google.cloud import bigquery_connection_v1

# Connection configuration that allows BigQuery to call Vertex AI embeddings.
CONNECTION_LOCATION = "us"
CONNECTION_ID = "vertex_ai_text_embeddings"
connection_parent = f"projects/{PROJECT_ID}/locations/{CONNECTION_LOCATION}"
connection_name = f"{connection_parent}/connections/{CONNECTION_ID}"

connection_client = bigquery_connection_v1.ConnectionServiceClient()

# Reuse the connection when it already exists, otherwise create a new Vertex AI link.
try:
  connection = connection_client.get_connection(name=connection_name)
  print(f"Connection `{connection.name}` already exists.")
except NotFound:
  conn = bigquery_connection_v1.types.Connection(
      cloud_resource=bigquery_connection_v1.types.CloudResourceProperties()
  )
  connection = connection_client.create_connection(
      parent=connection_parent,
      connection_id=CONNECTION_ID,
      connection=conn,
  )
  print(f"Created connection `{connection.name}` for Vertex AI embeddings.")


Created connection `projects/376838992535/locations/us/connections/vertex_ai_text_embeddings` for Vertex AI embeddings.


In [26]:
# Define a remote BigQuery ML model that proxies requests to the Vertex AI text-embedding-005 endpoint.
remote_model_sql = f"""
CREATE OR REPLACE MODEL `{PROJECT_ID}.{DATASET_ID}.embedding_model`
REMOTE WITH CONNECTION `{DATASET_LOCATION}.{CONNECTION_ID}`
OPTIONS (
  endpoint = 'text-embedding-005'
)
"""

# Execute the DDL so the model can be used in subsequent ML.GENERATE_TEXT_EMBEDDING calls.
query_job = bq_client.query(remote_model_sql)
query_job.result()
print(f"Created or updated remote model `{PROJECT_ID}.{DATASET_ID}.embedding_model` targeting text-embedding-005.")


Created or updated remote model `qwiklabs-gcp-04-ee8165cd97c8.genai_skills_workshop.embedding_model` targeting text-embedding-005.


In [27]:
# Inspect the newly loaded table to understand available fields for downstream processing.
table = bq_client.get_table(table_id)
print("Schema for aurora_bay_faqs:")
for field in table.schema:
  print(f"  {field.name}: {field.field_type}")


Schema for aurora_bay_faqs:
  string_field_0: STRING
  string_field_1: STRING


In [41]:
# Materialize a table that stores concatenated Q/A text alongside its embedding vector.
EMBEDDING_TABLE_ID = "aurora_bay_faqs_with_embeddings"
embedding_table_ref = f"{PROJECT_ID}.{DATASET_ID}.{EMBEDDING_TABLE_ID}"

embedding_sql = f"""
CREATE OR REPLACE TABLE `{embedding_table_ref}` AS
SELECT
  *
FROM
  ML.GENERATE_TEXT_EMBEDDING(
    MODEL `{DATASET_ID}.embedding_model`,
    (SELECT string_field_0 as question, string_field_1 as answer, CONCAT(string_field_0, ': ', string_field_0) AS content FROM {PROJECT_ID}.{DATASET_ID}.{TABLE_ID})
  );
"""


# Run the transformation query and wait for completion.
embedding_job = bq_client.query(embedding_sql)
embedding_job.result()
print(f"Created table `{embedding_table_ref}` with concatenated QA embeddings.")


Created table `qwiklabs-gcp-04-ee8165cd97c8.genai_skills_workshop.aurora_bay_faqs_with_embeddings` with concatenated QA embeddings.


In [34]:
import vertexai
from vertexai.generative_models import GenerativeModel, Part

def answer_question_gemini(prompt):
  """Invoke Gemini with consistent generation settings to answer a prompt."""
  model = GenerativeModel("gemini-2.5-flash-lite")
  response = model.generate_content(
    prompt,
    generation_config={
        "max_output_tokens": 8192,
        "temperature": 0.5,
        "top_p": 0.5,
        "top_k": 10,
    },
  stream=False,
  )
  try:
    return response.text
  except:
    print("An Error Ocuured Cleaning the Data")
    return "An Error Ocuured Cleaning the Data"

In [45]:
def run_search(question):
  from google.cloud import bigquery

  client = bigquery.Client()

  # Perform a vector search over the FAQ embedding table, using the remote model
  # to embed the incoming natural language question on the fly.
  sql = f"""
      SELECT base.question, base.answer
      FROM VECTOR_SEARCH(
      TABLE `{embedding_table_ref}`, 'text_embedding',
      (
      SELECT text_embedding, content AS query
      FROM ML.GENERATE_TEXT_EMBEDDING(MODEL `{PROJECT_ID}.{DATASET_ID}.embedding_model`,
          (SELECT @question AS content))),
      top_k => 5)
      """

  # Bind the user-entered question as a parameter to avoid SQL injection and reuse cached plans.
  job_config = bigquery.QueryJobConfig(
    query_parameters=[
        bigquery.ScalarQueryParameter("question", "STRING", question),
    ]
  )

  query_job = client.query(sql, job_config=job_config)

  # Format retrieved Q/A pairs as plain text paragraphs for downstream prompting.
  rows = []
  for row in query_job:
    rows.append(f"Q: {row.question}\nA: {row.answer}")

  return "\n\n".join(rows)

In [36]:
def build_prompt(data, question):
  """Wrap retrieved context in a simple instruction-following prompt."""
  prompt = """
    Instructions: Answer the question using the following Context.

    Context: {0}

    Question: {1}
  """.format(data, question)
  return prompt

In [37]:
from IPython.core.display import display, HTML

def answer_question(question):
  """Retrieve FAQ context, expose it inline, and ask Gemini for a final answer."""

  data = run_search(question)
  display("Retrieved Data:")
  display(data)
  display(" . . . ")
  prompt = build_prompt(data, question)
  answer_gemini = answer_question_gemini(prompt)

  return answer_gemini

In [46]:
QUESTION = "What are some fun activities in aurora?"

answer_gemini = answer_question(QUESTION)
display("User Question:")
display(QUESTION)
display("--------------------------------")
display("Gemini Answer:")
display(answer_gemini)

'Retrieved Data:'

'Q: What types of recreation are available in Aurora Bay?\nA: Popular activities include fishing, kayaking, hiking in the nearby forests, and northern lights viewing in the winter.\n\nQ: What cultural events are unique to Aurora Bay?\nA: Besides the Aurora Lights Winter Festival, there is the Seafarers’ Gathering in August, celebrating local fishing traditions with boat parades and seafood cook-offs.\n\nQ: What local outdoor adventure companies operate in Aurora Bay?\nA: North Star Excursions and Bay Explorers offer guided hikes, kayak tours, and wilderness camping experiences for visitors and residents.\n\nQ: What is the Aurora Lights Winter Festival?\nA: It’s Aurora Bay’s annual winter celebration featuring ice carving competitions, northern lights viewing tours, live music, and local food vendors. It typically takes place in late January.\n\nQ: Are there any hotels or lodging options in Aurora Bay?\nA: Yes. Options include the Aurora Bay Lodge (near the harbor), several bed-and-brea

' . . . '

/usr/local/lib/python3.12/dist-packages/vertexai/generative_models/_generative_models.py:433: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()


'User Question:'

'What are some fun activities in aurora?'

'--------------------------------'

'Gemini Answer:'

'Based on the context provided, some fun activities in Aurora Bay include:\n\n*   Fishing\n*   Kayaking\n*   Hiking in the nearby forests\n*   Northern lights viewing (especially in winter)\n*   Attending the Aurora Lights Winter Festival (which features ice carving, live music, and local food)\n*   Participating in the Seafarers’ Gathering (with boat parades and seafood cook-offs)\n*   Guided hikes, kayak tours, and wilderness camping experiences offered by local companies.'